# Function Calling with GPT Models

This notebook explores the integration of GPT models with external functions. We'll demonstrate how to call functions within the GPT context, convert Python functions to a JSON-compatible format, and execute functions retrieved from GPT outputs.

### Content Overview:

**I. [Understanding Function Calls in GPT](#section_one)** </br>
>- Explore how GPT models can interpret and utilize JSON-formatted functions.
>- Learn about the necessary components of a JSON function for GPT compatibility.

**II. [Python to JSON Function Conversion](#section_two)** </br>
>- Step-by-step guide to transforming a Python function into a JSON format.
>- Methods for passing JSON functions to a GPT model for processing.

**III. [Executing Functions from GPT Outputs](#section_three)** </br>
>- Detailed example on how to retrieve a function from GPT output and execute it in Python.
>- A good practice for handling and validating the execution of functions from GPT.

**References:**
>- https://github.com/fastai/lm-hackers/blob/main/lm-hackers.ipynb by Jeremy Howard
>- https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling?tabs=python

**NOTE:**
- This code requires pydantic==2.5.1 for successful exection.

### OpenAI chat completion arguments:

Below is a sample code snippet that demonstrates how to use the OpenAI Chat Completion API with function calling:

```
response = openai.ChatCompletion.create(
    engine=gpt_model, # Name of the gpt model : str
    messages=messages, # Input messages: List[Dict]
    functions=search_hotels_function_json, # List of functions in JSON format: List
    function_call="auto", # 'auto': GPT model decides when to use functions. None: GPT model will not use the functions. Dict {"name": "function_name"}: GPT model will always use the defined functions.
    temperature=temperature # 0 to 1: float
)
```

In [4]:
# import libraries
from pydantic import create_model
import inspect, json
from inspect import Parameter
import openai
import os
from dotenv import load_dotenv

load_dotenv()

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [22]:
# define the necessary configs
gpt_model = "gpt-35-turbo-16k"
temperature = 0

<a id=section_one></a>
**Understanding Function Calls in GPT**

In [23]:
search_hotels_function_json = [  
    {
        "name": "search_hotels",
        "description": "Retrieves hotels from the search index based on the parameters provided",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The location of the hotel (i.e. Seattle, WA)"
                },
                "max_price": {
                    "type": "number",
                    "description": "The maximum price for the hotel"
                },
                "features": {
                    "type": "string",
                    "description": "A comma separated list of features (i.e. beachfront, free wifi, etc.)"
                }
            },
            "required": ["location"]
        }
    }
]

messages= [
    {"role": "user", "content": "Find beachfront hotels in San Diego for less than $300 a month with free breakfast."}
]

# Pass th function toGPT model
response = openai.ChatCompletion.create(
    engine=gpt_model,
    messages=messages,
    functions=search_hotels_function_json,
    function_call="auto",
    temperature=temperature
)

In [24]:
print("Full response:")
print(response.choices[0].message, "\n")
print("\nFunction call:")
print(response.choices[0].message.function_call, "\n")
print("\nFunction call arguments:")
print(response.choices[0].message.function_call.arguments, "\n")

Full response:
{
  "role": "assistant",
  "function_call": {
    "name": "search_hotels",
    "arguments": "{\n  \"location\": \"San Diego\",\n  \"max_price\": 300,\n  \"features\": \"beachfront,free breakfast\"\n}"
  }
} 


Function call:
{
  "name": "search_hotels",
  "arguments": "{\n  \"location\": \"San Diego\",\n  \"max_price\": 300,\n  \"features\": \"beachfront,free breakfast\"\n}"
} 


Function call arguments:
{
  "location": "San Diego",
  "max_price": 300,
  "features": "beachfront,free breakfast"
} 



<a id=section_two></a>
**Python to JSON Function Conversion**

In [28]:
def sum(a:int, b:int=1):
    "Adds a + b"
    return a + b

def jsonschema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

print(type(jsonschema(sum)), "\n")
jsonschema(sum)

<class 'dict'> 



C:\Users\froozitalab\AppData\Local\Temp\ipykernel_5244\2071725937.py:8: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


{'name': 'sum',
 'description': 'Adds a + b',
 'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
   'b': {'default': 1, 'title': 'B', 'type': 'integer'}},
  'required': ['a'],
  'title': 'Input for `sum`',
  'type': 'object'}}

When functions are provided, by default the function_call will be set to "auto" and the model will decide whether or not a function should be called. 

In [31]:
llm_system_role = "Answer the user's question."
prompt = "What is 6+3?"

response = openai.ChatCompletion.create(
            engine=gpt_model,
            messages=[
                {"role": "system", "content": llm_system_role},
                {"role": "user", "content": prompt}
            ],
            temperature=temperature,
            functions=[jsonschema(sum)],
            function_call={"name": "sum"}
        )

C:\Users\froozitalab\AppData\Local\Temp\ipykernel_5244\2071725937.py:8: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


In [32]:
print("Full response:")
print(response.choices[0].message, "\n")
print("\nFunction call:")
print(response.choices[0].message.function_call, "\n")
print("\nFunction call arguments:")
print(response.choices[0].message.function_call.arguments, "\n")

Full response:
{
  "role": "assistant",
  "function_call": {
    "name": "sum",
    "arguments": "{\n  \"a\": 6,\n  \"b\": 3\n}"
  }
}

Function call:
{
  "name": "sum",
  "arguments": "{\n  \"a\": 6,\n  \"b\": 3\n}"
}

Function call arguments:
{
  "a": 6,
  "b": 3
}


In [35]:
funcs_ok = {'sums', 'python'}

def call_func(response):
    function = response.choices[0].message.function_call
    if function.name not in funcs_ok: return print(f'Not allowed: {function.name}')
    f = globals()[function.name]
    return f(**json.loads(function.arguments))

call_func(response)

9

In [11]:
print("Full response:")
print(response.choices[0].message)
print("\nFunction call arguments:")
print(response.choices[0].message.function_call.arguments)

Full response:
{
  "role": "assistant",
  "function_call": {
    "name": "sums",
    "arguments": "{\n  \"a\": 6,\n  \"b\": 3\n}"
  }
}

Function call arguments:
{
  "a": 6,
  "b": 3
}


<a id=section_three></a>
**Executing Functions from GPT Outputs**

In [36]:
import ast

def run(code):
    """
    Executes the given Python code and returns the result of the last expression.
    Args:
        code (str): The Python code to execute.
    Returns:
        The result of the last expression in the code, or None if there is no expression.
    Raises:
        SyntaxError: If the code contains syntax errors.
        TypeError: If the code is not a string.
    """
    tree = ast.parse(code)
    last_node = tree.body[-1] if tree.body else None
    
    # If the last node is an expression, modify the AST to capture the result
    if isinstance(last_node, ast.Expr):
        tgts = [ast.Name(id='_result', ctx=ast.Store())]
        assign = ast.Assign(targets=tgts, value=last_node.value)
        tree.body[-1] = ast.fix_missing_locations(assign)

    ns = {}
    exec(compile(tree, filename='<ast>', mode='exec'), ns)
    return ns.get('_result', None)

def python(code:str):
    "Return result of executing `code` using python. If execution not permitted, returns `#FAIL#`"
    go = input(f'Proceed with execution?\n```\n{code}\n```\n')
    if go.lower()!='y': return '#FAIL#'
    return run(code)

In [38]:
run("""
a=2
b=4
a**b
""")

16

In [39]:
prompt = "What is 12 factorial?"
llm_system_role = "You are a math expert. Use your best knowledge to answer the question."

response = openai.ChatCompletion.create(
            engine=GPT_MODEL,
            messages=[
                {"role": "system", "content": llm_system_role},
                {"role": "user", "content": prompt}
            ],
            temperature=TEMPERATURE,
        )

print("Full response:")
print(response.choices[0].message.content)

Full response:
12 factorial, denoted as 12!, is the product of all positive integers from 1 to 12. Mathematically, it can be calculated as:

12! = 12 × 11 × 10 × 9 × 8 × 7 × 6 × 5 × 4 × 3 × 2 × 1

Simplifying this expression, we get:

12! = 479,001,600

Therefore, 12 factorial is equal to 479,001,600.


Given result: 479,001,600

In [42]:
prompt = "What is 12 factorial?"
llm_system_role = "Use python for any required computations."

response = openai.ChatCompletion.create(
            engine=GPT_MODEL,
            messages=[
                {"role": "system", "content": llm_system_role},
                {"role": "user", "content": prompt}
            ],
            temperature=TEMPERATURE,
            functions=[jsonschema(python)],
            function_call="auto"
        )

print("Full response:")
print(response.choices[0].message)
print("\nFunction call arguments:")
print(response.choices[0].message.function_call.arguments)

Full response:
{
  "role": "assistant",
  "function_call": {
    "name": "python",
    "arguments": "{\n  \"code\": \"import math\\nmath.factorial(12)\"\n}"
  }
}

Function call arguments:
{
  "code": "import math\nmath.factorial(12)"
}


In [45]:
response

<OpenAIObject chat.completion id=chatcmpl-8MJytC90aoWYqWOjZmztdWDBBctnT at 0x16c86333470> JSON: {
  "id": "chatcmpl-8MJytC90aoWYqWOjZmztdWDBBctnT",
  "object": "chat.completion",
  "created": 1700330715,
  "model": "gpt-35-turbo-16k",
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "choices": [
    {
      "index": 0,
      "finish_reason": "function_call",
      "message": {
        "role": "assistant",
        "function_call": {
          "name": "python",
          "arguments": "{\n  \"code\": \"import math\\nmath.factorial(12)\"\n}"
        }
      },
      "conten

In [47]:
call_func(response)

479001600